In [1]:
import sqlite3
import time
import pandas as pd
import numpy as np
import query

from datetime import datetime

종목코드열람

In [2]:
shcode_list = query.query_shcode_list_today()

In [3]:
# array_df: numpy array date, open, high, low, close, volume
def fun_incresing_ratio_test(array_df, limit = 0.15):
    #1) 종가비율 limit 이상
    ratio_list = [y/x-1 for x, y in zip(array_df[:,4][:-1], array_df[:,4][1:])]
    ratio_test = np.array([i >= limit for i in ratio_list])
    ratio_test = np.insert(ratio_test, 0 , 0)
    return ratio_test

In [4]:
def fun_total_volume_test(array_df, min_total_volume = 30000000000):
    
    #2) 해당날 거래대금 300억 이상 거래대금 = 종가 * 거래량
    total_volume_list = array_df[:,4] * array_df[:,5]
    total_volume_test = [i >= min_total_volume  for i in total_volume_list]
    total_volume_test = np.array(total_volume_test[:])
    return total_volume_test


In [5]:
def fun_average_volume_test(array_df, ratio_test, total_volume_test):
    #3) 급등시점 전 평균 20일간 거래량의 1000%
    average_volume_list = ratio_test * total_volume_test
    average_volume_list_index = [ i for i,v in enumerate(average_volume_list) if v == True]
    
    average_volume_test = np.array([0 for i in range(len(average_volume_list))])
    
    for ind in average_volume_list_index:
        if ind < 20:
            average_vol =  array_df[:,5][:ind].sum() / ind
        else:
            average_vol =  array_df[:,5][ind-20:ind].sum() / 20
        if array_df[:,5][ind] >= average_vol * 10:
            average_volume_test[ind] = 1
            
    return average_volume_test

#최종

In [ ]:
result = pd.DataFrame([], columns= ['shcode',"index"])
high_ratio = 0.15
volume_money = 30000000000

for i, v in enumerate(shcode_list['shcode']):
    
    now = datetime.now()
    now_time = '%s-%s-%s_%s:%s:%s' % ( now.year, now.month, now.day, now.hour, now.minute, now.second)
    print('------------------- test 시작_%s : , %s , %s , %s -------------------  ' %(now_time, i, v, shcode_list['shname'].iloc[i]))
    
    daily_data = query.query_ohlcv(v, to ='20190927')
    array_daily = np.array(daily_data)
    
    ratio_test = fun_incresing_ratio_test(array_daily, limit = high_ratio)
    if any(ratio_test) != 1:
        print('X_1. ratio_test 없음 %s , %s , %s' %( i, v, shcode_list['shname'].iloc[i]))        
        continue
    print('O_1. ratio_test 통과 %s , %s , %s' %( i, v, shcode_list['shname'].iloc[i]))  
    
    total_volume_test = fun_total_volume_test(array_daily, min_total_volume = volume_money)
    if any(total_volume_test) != 1:
        print('X_2. total_volume_test 없음 %s , %s , %s' %( i, v, shcode_list['shname'].iloc[i]))         
        continue
    print('O_2. total_volume_test 통과 %s , %s , %s' %( i, v, shcode_list['shname'].iloc[i])) 
    
    average_volume_test = fun_average_volume_test(array_daily, ratio_test, total_volume_test)
    if any(average_volume_test) != 1:
        print('X_3. average_volume_test 없음 %s , %s , %s' %( i, v, shcode_list['shname'].iloc[i]))            
        continue
    print('O_3. average_volume_test 통과 %s , %s , %s' %( i, v, shcode_list['shname'].iloc[i]))     
    
    result_test = ratio_test * total_volume_test * average_volume_test
    final_index = array_daily[:,0][result_test==1]
    final_value = " ".join(str(x) for x in final_index)
    #final_index = np.where(result_test==1)
    #final_value = " ".join(str(x) for x in final_index[0])
    
    test_result = pd.DataFrame([[v,final_value]], columns= ['shcode',"index"])
    result = result.append(test_result, ignore_index = True)

    print('------------------- test 완료 : , %s , %s , %s ------------------- ' %(i, v, shcode_list['shname'].iloc[i]))

filename = 'high_ratio'+'_'+str(high_ratio)+'_' + 'volume_money'+'_'+str(volume_money)+ '_v2' +'.csv'
result.to_csv(filename, sep=',')

In [6]:
result = pd.DataFrame([], columns= ['shcode',"index"])
high_ratio = 0.10
volume_money = 20000000000

for i, v in enumerate(shcode_list['shcode']):
    
    now = datetime.now()
    now_time = '%s-%s-%s_%s:%s:%s' % ( now.year, now.month, now.day, now.hour, now.minute, now.second)
    print('------------------- test 시작_%s : , %s , %s , %s -------------------  ' %(now_time, i, v, shcode_list['shname'].iloc[i]))
    
    daily_data = query.query_ohlcv(v, to ='20190927')
    array_daily = np.array(daily_data)
    
    ratio_test = fun_incresing_ratio_test(array_daily, limit = high_ratio)
    if any(ratio_test) != 1:
        print('X_1. ratio_test 없음 %s , %s , %s' %( i, v, shcode_list['shname'].iloc[i]))        
        continue
    print('O_1. ratio_test 통과 %s , %s , %s' %( i, v, shcode_list['shname'].iloc[i]))  
    
    total_volume_test = fun_total_volume_test(array_daily, min_total_volume = volume_money)
    if any(total_volume_test) != 1:
        print('X_2. total_volume_test 없음 %s , %s , %s' %( i, v, shcode_list['shname'].iloc[i]))         
        continue
    print('O_2. total_volume_test 통과 %s , %s , %s' %( i, v, shcode_list['shname'].iloc[i])) 
    
    average_volume_test = fun_average_volume_test(array_daily, ratio_test, total_volume_test)
    if any(average_volume_test) != 1:
        print('X_3. average_volume_test 없음 %s , %s , %s' %( i, v, shcode_list['shname'].iloc[i]))            
        continue
    print('O_3. average_volume_test 통과 %s , %s , %s' %( i, v, shcode_list['shname'].iloc[i]))     
    
    result_test = ratio_test * total_volume_test * average_volume_test
    final_index = array_daily[:,0][result_test==1]
    final_value = " ".join(str(x) for x in final_index)
    #final_index = np.where(result_test==1)
    #final_value = " ".join(str(x) for x in final_index[0])
    
    test_result = pd.DataFrame([[v,final_value]], columns= ['shcode',"index"])
    result = result.append(test_result, ignore_index = True)

    print('------------------- test 완료 : , %s , %s , %s ------------------- ' %(i, v, shcode_list['shname'].iloc[i]))

filename = 'high_ratio'+'_'+str(high_ratio)+'_' + 'volume_money'+'_'+str(volume_money)+ '_v2' +'.csv'
result.to_csv(filename, sep=',')

------------------- test 시작_2019-11-6_16:12:36 : , 0 , 000020 , 동화약품 -------------------  
X_1. ratio_test 없음 0 , 000020 , 동화약품
------------------- test 시작_2019-11-6_16:12:37 : , 1 , 000040 , KR모터스 -------------------  
X_1. ratio_test 없음 1 , 000040 , KR모터스
------------------- test 시작_2019-11-6_16:12:38 : , 2 , 000050 , 경방 -------------------  
X_1. ratio_test 없음 2 , 000050 , 경방
------------------- test 시작_2019-11-6_16:12:38 : , 3 , 000060 , 메리츠화재 -------------------  
X_1. ratio_test 없음 3 , 000060 , 메리츠화재
------------------- test 시작_2019-11-6_16:12:39 : , 4 , 000070 , 삼양홀딩스 -------------------  
X_1. ratio_test 없음 4 , 000070 , 삼양홀딩스
------------------- test 시작_2019-11-6_16:12:39 : , 5 , 000075 , 삼양홀딩스우 -------------------  
X_1. ratio_test 없음 5 , 000075 , 삼양홀딩스우
------------------- test 시작_2019-11-6_16:12:40 : , 6 , 000080 , 하이트진로 -------------------  
X_1. ratio_test 없음 6 , 000080 , 하이트진로
------------------- test 시작_2019-11-6_16:12:40 : , 7 , 000087 , 하이트진로2우B -------------------  
X

X_1. ratio_test 없음 53 , 000880 , 한화
------------------- test 시작_2019-11-6_16:13:1 : , 54 , 000885 , 한화우 -------------------  
O_1. ratio_test 통과 54 , 000885 , 한화우
O_2. total_volume_test 통과 54 , 000885 , 한화우
O_3. average_volume_test 통과 54 , 000885 , 한화우
------------------- test 완료 : , 54 , 000885 , 한화우 ------------------- 
------------------- test 시작_2019-11-6_16:13:2 : , 55 , 00088K , 한화3우B -------------------  
X_1. ratio_test 없음 55 , 00088K , 한화3우B
------------------- test 시작_2019-11-6_16:13:2 : , 56 , 000890 , 보해양조 -------------------  
O_1. ratio_test 통과 56 , 000890 , 보해양조
O_2. total_volume_test 통과 56 , 000890 , 보해양조
O_3. average_volume_test 통과 56 , 000890 , 보해양조
------------------- test 완료 : , 56 , 000890 , 보해양조 ------------------- 
------------------- test 시작_2019-11-6_16:13:3 : , 57 , 000910 , 유니온 -------------------  
O_1. ratio_test 통과 57 , 000910 , 유니온
O_2. total_volume_test 통과 57 , 000910 , 유니온
O_3. average_volume_test 통과 57 , 000910 , 유니온
------------------- test 완료 : , 57 

X_1. ratio_test 없음 102 , 001560 , 제일연마
------------------- test 시작_2019-11-6_16:13:24 : , 103 , 001570 , 금양 -------------------  
X_1. ratio_test 없음 103 , 001570 , 금양
------------------- test 시작_2019-11-6_16:13:25 : , 104 , 001620 , 케이비아이동국실업 -------------------  
O_1. ratio_test 통과 104 , 001620 , 케이비아이동국실업
X_2. total_volume_test 없음 104 , 001620 , 케이비아이동국실업
------------------- test 시작_2019-11-6_16:13:25 : , 105 , 001630 , 종근당홀딩스 -------------------  
O_1. ratio_test 통과 105 , 001630 , 종근당홀딩스
X_2. total_volume_test 없음 105 , 001630 , 종근당홀딩스
------------------- test 시작_2019-11-6_16:13:26 : , 106 , 001680 , 대상 -------------------  
X_1. ratio_test 없음 106 , 001680 , 대상
------------------- test 시작_2019-11-6_16:13:26 : , 107 , 001685 , 대상우 -------------------  
X_1. ratio_test 없음 107 , 001685 , 대상우
------------------- test 시작_2019-11-6_16:13:27 : , 108 , 001720 , 신영증권 -------------------  
X_1. ratio_test 없음 108 , 001720 , 신영증권
------------------- test 시작_2019-11-6_16:13:27 : , 109 , 001725 , 

O_1. ratio_test 통과 146 , 002450 , 삼익악기
O_2. total_volume_test 통과 146 , 002450 , 삼익악기
O_3. average_volume_test 통과 146 , 002450 , 삼익악기
------------------- test 완료 : , 146 , 002450 , 삼익악기 ------------------- 
------------------- test 시작_2019-11-6_16:13:44 : , 147 , 002460 , 화성산업 -------------------  
X_1. ratio_test 없음 147 , 002460 , 화성산업
------------------- test 시작_2019-11-6_16:13:44 : , 148 , 002600 , 조흥 -------------------  
X_1. ratio_test 없음 148 , 002600 , 조흥
------------------- test 시작_2019-11-6_16:13:45 : , 149 , 002620 , 제일파마홀딩스 -------------------  
X_1. ratio_test 없음 149 , 002620 , 제일파마홀딩스
------------------- test 시작_2019-11-6_16:13:45 : , 150 , 002630 , 오리엔트바이오 -------------------  
O_1. ratio_test 통과 150 , 002630 , 오리엔트바이오
O_2. total_volume_test 통과 150 , 002630 , 오리엔트바이오
O_3. average_volume_test 통과 150 , 002630 , 오리엔트바이오
------------------- test 완료 : , 150 , 002630 , 오리엔트바이오 ------------------- 
------------------- test 시작_2019-11-6_16:13:46 : , 151 , 002690 , 동일제강 -----------

O_1. ratio_test 통과 189 , 003415 , 쌍용양회우
X_2. total_volume_test 없음 189 , 003415 , 쌍용양회우
------------------- test 시작_2019-11-6_16:14:4 : , 190 , 003460 , 유화증권 -------------------  
X_1. ratio_test 없음 190 , 003460 , 유화증권
------------------- test 시작_2019-11-6_16:14:4 : , 191 , 003465 , 유화증권우 -------------------  
X_1. ratio_test 없음 191 , 003465 , 유화증권우
------------------- test 시작_2019-11-6_16:14:5 : , 192 , 003470 , 유안타증권 -------------------  
X_1. ratio_test 없음 192 , 003470 , 유안타증권
------------------- test 시작_2019-11-6_16:14:5 : , 193 , 003475 , 유안타증권우 -------------------  
X_1. ratio_test 없음 193 , 003475 , 유안타증권우
------------------- test 시작_2019-11-6_16:14:6 : , 194 , 003480 , 한진중공업홀딩스 -------------------  
O_1. ratio_test 통과 194 , 003480 , 한진중공업홀딩스
X_2. total_volume_test 없음 194 , 003480 , 한진중공업홀딩스
------------------- test 시작_2019-11-6_16:14:6 : , 195 , 003490 , 대한항공 -------------------  
X_1. ratio_test 없음 195 , 003490 , 대한항공
------------------- test 시작_2019-11-6_16:14:6 : , 196 , 00349

...

O_1. ratio_test 통과 2717 , 234100 , 세원
O_2. total_volume_test 통과 2717 , 234100 , 세원
O_3. average_volume_test 통과 2717 , 234100 , 세원
------------------- test 완료 : , 2717 , 234100 , 세원 ------------------- 
------------------- test 시작_2019-11-6_16:32:3 : , 2718 , 234300 , 에스트래픽 -------------------  
O_1. ratio_test 통과 2718 , 234300 , 에스트래픽
O_2. total_volume_test 통과 2718 , 234300 , 에스트래픽
O_3. average_volume_test 통과 2718 , 234300 , 에스트래픽
------------------- test 완료 : , 2718 , 234300 , 에스트래픽 ------------------- 
------------------- test 시작_2019-11-6_16:32:3 : , 2719 , 234340 , 세틀뱅크 -------------------  
X_1. ratio_test 없음 2719 , 234340 , 세틀뱅크
------------------- test 시작_2019-11-6_16:32:3 : , 2720 , 234920 , 자이글 -------------------  
O_1. ratio_test 통과 2720 , 234920 , 자이글
X_2. total_volume_test 없음 2720 , 234920 , 자이글
------------------- test 시작_2019-11-6_16:32:4 : , 2721 , 236200 , 슈프리마 -------------------  
O_1. ratio_test 통과 2721 , 236200 , 슈프리마
O_2. total_volume_test 통과 2721 , 236200 , 슈프리마


O_1. ratio_test 통과 2755 , 252500 , 세화피앤씨
O_2. total_volume_test 통과 2755 , 252500 , 세화피앤씨
O_3. average_volume_test 통과 2755 , 252500 , 세화피앤씨
------------------- test 완료 : , 2755 , 252500 , 세화피앤씨 ------------------- 
------------------- test 시작_2019-11-6_16:32:19 : , 2756 , 253450 , 스튜디오드래곤 -------------------  
X_1. ratio_test 없음 2756 , 253450 , 스튜디오드래곤
------------------- test 시작_2019-11-6_16:32:19 : , 2757 , 253590 , 네오셈 -------------------  
O_1. ratio_test 통과 2757 , 253590 , 네오셈
O_2. total_volume_test 통과 2757 , 253590 , 네오셈
O_3. average_volume_test 통과 2757 , 253590 , 네오셈
------------------- test 완료 : , 2757 , 253590 , 네오셈 ------------------- 
------------------- test 시작_2019-11-6_16:32:19 : , 2758 , 253840 , 수젠텍 -------------------  
O_1. ratio_test 통과 2758 , 253840 , 수젠텍
O_2. total_volume_test 통과 2758 , 253840 , 수젠텍
O_3. average_volume_test 통과 2758 , 253840 , 수젠텍
------------------- test 완료 : , 2758 , 253840 , 수젠텍 ------------------- 
------------------- test 시작_2019-11-6_16:32:20 :

X_1. ratio_test 없음 2792 , 264850 , IBKS제6호스팩
------------------- test 시작_2019-11-6_16:32:34 : , 2793 , 265480 , 미래에셋대우스팩1호 -------------------  
X_1. ratio_test 없음 2793 , 265480 , 미래에셋대우스팩1호
------------------- test 시작_2019-11-6_16:32:35 : , 2794 , 265520 , AP시스템 -------------------  
X_1. ratio_test 없음 2794 , 265520 , AP시스템
------------------- test 시작_2019-11-6_16:32:35 : , 2795 , 265560 , 영화테크 -------------------  
O_1. ratio_test 통과 2795 , 265560 , 영화테크
X_2. total_volume_test 없음 2795 , 265560 , 영화테크
------------------- test 시작_2019-11-6_16:32:36 : , 2796 , 265920 , 한화수성스팩 -------------------  
X_1. ratio_test 없음 2796 , 265920 , 한화수성스팩
------------------- test 시작_2019-11-6_16:32:36 : , 2797 , 267320 , 교보7호스팩 -------------------  
X_1. ratio_test 없음 2797 , 267320 , 교보7호스팩
------------------- test 시작_2019-11-6_16:32:37 : , 2798 , 267790 , 배럴 -------------------  
X_1. ratio_test 없음 2798 , 267790 , 배럴
------------------- test 시작_2019-11-6_16:32:37 : , 2799 , 267980 , 매일유업 --------------

X_1. ratio_test 없음 2834 , 291230 , 삼성스팩2호
------------------- test 시작_2019-11-6_16:32:52 : , 2835 , 293580 , 나우아이비캐피탈 -------------------  
X_1. ratio_test 없음 2835 , 293580 , 나우아이비캐피탈
------------------- test 시작_2019-11-6_16:32:53 : , 2836 , 293780 , 압타바이오 -------------------  
O_1. ratio_test 통과 2836 , 293780 , 압타바이오
O_2. total_volume_test 통과 2836 , 293780 , 압타바이오
X_3. average_volume_test 없음 2836 , 293780 , 압타바이오
------------------- test 시작_2019-11-6_16:32:53 : , 2837 , 297570 , IBKS제9호스팩 -------------------  
O_1. ratio_test 통과 2837 , 297570 , IBKS제9호스팩
O_2. total_volume_test 통과 2837 , 297570 , IBKS제9호스팩
O_3. average_volume_test 통과 2837 , 297570 , IBKS제9호스팩
------------------- test 완료 : , 2837 , 297570 , IBKS제9호스팩 ------------------- 
------------------- test 시작_2019-11-6_16:32:54 : , 2838 , 298380 , 에이비엘바이오 -------------------  
O_1. ratio_test 통과 2838 , 298380 , 에이비엘바이오
O_2. total_volume_test 통과 2838 , 298380 , 에이비엘바이오
X_3. average_volume_test 없음 2838 , 298380 , 에이비엘바이오
-----------

X_1. ratio_test 없음 2876 , 320000 , 하나금융13호스팩
------------------- test 시작_2019-11-6_16:33:10 : , 2877 , 321260 , 유진스팩4호 -------------------  
O_1. ratio_test 통과 2877 , 321260 , 유진스팩4호
O_2. total_volume_test 통과 2877 , 321260 , 유진스팩4호
X_3. average_volume_test 없음 2877 , 321260 , 유진스팩4호
------------------- test 시작_2019-11-6_16:33:10 : , 2878 , 322780 , DB금융스팩7호 -------------------  
O_1. ratio_test 통과 2878 , 322780 , DB금융스팩7호
O_2. total_volume_test 통과 2878 , 322780 , DB금융스팩7호
X_3. average_volume_test 없음 2878 , 322780 , DB금융스팩7호
------------------- test 시작_2019-11-6_16:33:11 : , 2879 , 323210 , 이베스트이안스팩1호 -------------------  
O_1. ratio_test 통과 2879 , 323210 , 이베스트이안스팩1호
O_2. total_volume_test 통과 2879 , 323210 , 이베스트이안스팩1호
X_3. average_volume_test 없음 2879 , 323210 , 이베스트이안스팩1호
------------------- test 시작_2019-11-6_16:33:11 : , 2880 , 323230 , 신한제5호스팩 -------------------  
O_1. ratio_test 통과 2880 , 323230 , 신한제5호스팩
O_2. total_volume_test 통과 2880 , 323230 , 신한제5호스팩
X_3. average_volume_test 없음